In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from asbe.base import *
from asbe.models import *
from asbe.estimators import *
from dataclasses import dataclass
import numpy as np
from collections.abc import Callable
from typing import Union

In [ ]:
# Data generating process

In [ ]:
@dataclass
class DataGenerator():
    ds: str = None
    dgp_x : Union[Callable, None] = None
    dgp_t : Union[Callable, None] = None
    dgp_y : Union[Callable, None] = None
    
    def get_X(self, method : str = "random", no_query : int = 1):
        if self.ds is not None:
            ix = np.random.randint(low = 0, high = self.ds["X_pool"].shape[0], size=(no_query))
            X_new = self.ds["X_pool"][ix,:]
            self.selected_ix = ix
        else:
            X_new = self.dgp_x()
        self.X_new = X_new
        return X_new
            
    def get_t(self, X_new):
        if X_new is None:
            X_new = self.X_new
        if self.ds is not None:
            t_new = self.ds["t_pool"][self.selected_ix]
        else:
            t_new = self.dgp_t(X_new)
        self.t_new = t_new

In [ ]:
dg = DataGenerator(ds = {"X_pool":np.array([[1,2,3,4,5,6],[7,8,9,0,1,2]])})

In [ ]:
dg.get_X()

array([[7, 8, 9, 0, 1, 2]])